In [4]:
%matplotlib notebook

from __future__ import division
from __future__ import print_function

# Required modules
import numpy as np
import matplotlib.pyplot as plt
import warnings

from hmmlearn.hmm import GaussianHMM, GMMHMM
from hmmlearn.utils import iter_from_X_lengths

from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import zero_one_loss
from sklearn.utils import shuffle

from scipy.io import loadmat

import scipy.io as sio   

matfn = 'trajectories_train.mat'
data = sio.loadmat(matfn)  


In [5]:
for i in data:
    print(i)
data['xtrain']

__header__
__version__
__globals__
key
xtrain
ytrain


In [136]:
len(data['xtrain'][0]) #1429 * 3 * 174
lengths = list(map(len, [i[0] for i in data['xtrain'][0]]))

# get the index of every different charactors.

def get_data(num):
    return data['xtrain'][data['ytrain'] == num]

# for i in range(0,26):
#     print(len(get_index(i)))


In [85]:
#init guess for EM
# transition matrix for latent variable

n_states = 20

trans0 = np.diag(np.ones(n_states)) + np.diag(np.ones(n_states-1), 1)

trans0 /= trans0.sum(axis=1).reshape(-1, 1)
trans0                

array([[ 0.5,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.

In [128]:
X_train = {}
X_len = {}
for i in range(1, 21):
    X_train[i] = np.vstack(get_data(i))
    X_len[i] = list(map(len, [j[0] for j in get_data(i)]))
    
len(get_data(1))

83

In [140]:
# X_train[1][12]
# print(len(X_train[1][12]))
# print(X_len[2])
# for i in X_len:
#     print(X_len[i])
    
a = set(len(i[0]) for i in data['xtrain'][0])
len(data['ytrain'][0])

1429

In [141]:
train_x = data['xtrain'][0]
train_y = data['ytrain'][0]

In [142]:
len(X_train[4])

213

In [144]:
hmm = {}
for i in range(1, 21):
    hmm[i] = GaussianHMM(n_components=n_states, 
                   init_params='mc',
                   n_iter=10,
                   random_state=2)
 

In [ ]:
pi0 = 